In [2]:
!pip install pyarabic
!git clone https://github.com/moaaztaha/Arabic-English-Translation-Transformers

     |████████████████████████████████| 112kB 8.8MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=a60674902d7fdf54ac8084e12405eabd5b1c2a7a7931e22614c79372022a5f96
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'Arabic-English-Translation-Transformers'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 46 (delta 16), reused 40 (delta 10), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [22]:
# modules
import random
import string
import re
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import pandas as pd
from tqdm import tqdm

import gensim
import numpy as np

import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, strip_tatweel

### Data Preprocessing 

In [4]:
ar = pd.read_table('/content/Arabic-English-Translation-Transformers/ArabicNewData.txt', delimiter='\\n', names=['ar'])
en = pd.read_table('/content/Arabic-English-Translation-Transformers/EnglishNewData.txt', delimiter='\\n', names=['en'])

en['ar'] = ar['ar']
df = en.copy()
df = df.iloc[:35118]

/usr/local/lib/python3.7/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [5]:
morphs = [strip_tashkeel, strip_tatweel]

def fix_ar(sent):
  sent = split_al_sent(sent)
  tokens = araby.tokenize(sent, morphs=morphs)
  sent = araby.normalize_hamza(' '.join(tokens), method='tasheel')
  return sent

In [6]:
def split_al(word):
    if word.startswith('ال'):
        return word[:2], word[2:]
    else: 
        return word

def split_al_sent(sent):
    ww = []
    for word in sent.split():
        out = split_al(word)
        if type(out) is tuple:
            for w in out:
                ww.append(w)
        else:
            ww.append(word)
    return ' '.join(w for w in ww)

In [7]:
df['ar'] = df.apply(lambda row: fix_ar(row.ar), axis=1)

In [8]:
text_pairs = []
for idx, row in df.iterrows():
  en, ar = row['en'], row['ar']
  ar = "[start] " + ar + " [end]"
  text_pairs.append((en, ar))

In [9]:
for idx, row in df.iterrows():
  if len(row.ar.split()) < 1:
    print(row.ar, '\n*')
    print(row.en)

In [10]:
for _ in range(2):
    print(random.choice(text_pairs))

('as promised...?', '[start] {\\ 3cH5A1867 } كما وعدتك يازيرو ...؟ [end]')
('Cheetah, come back!', '[start] شيتا ، يرجع ! [end]')


In [11]:
len(text_pairs)

35118

In [12]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[: num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]

In [13]:
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

35118 total pairs
29851 training pairs
5267 validation pairs


#### Vectorizing the text data 

In [14]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


vocab_size = 20000
sequence_length = 50
batch_size = 265

def custom_standardization(input_string):
    return tf.strings.regex_replace(input_string, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(
            # max_tokens=vocab_size, 
            output_mode='int', 
            output_sequence_length=sequence_length)

ar_vectorization = TextVectorization(
    # max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    max_tokens=vocab_size)

eng_texts = [pair[0] for pair in text_pairs]
ar_texts = [pair[1] for pair in text_pairs]
eng_vectorization.adapt(eng_texts)
ar_vectorization.adapt(ar_texts)

In [15]:
len(ar_vectorization.get_vocabulary()), len(eng_vectorization.get_vocabulary())

(20000, 13164)

In [16]:
len(ar_vectorization.get_vocabulary()), len(eng_vectorization.get_vocabulary())

(20000, 13164)

In [17]:
def format_dataset(eng, ar):
    eng = eng_vectorization(eng)
    ar = ar_vectorization(ar)
    return ({"encoder_inputs": eng, "decoder_inputs": ar[:, :-1],}, ar[:, 1:])


def make_dataset(pairs):
    eng_texts, ar_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ar_texts = list(ar_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ar_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [18]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [19]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (265, 50)
inputs["decoder_inputs"].shape: (265, 50)
targets.shape: (265, 50)


### Building the Model 

In [20]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):

      config = super().get_config().copy()
      config.update({
          'embed_dim': self.embed_dim,
          'dense_dim': self.dense_dim,
          'num_heads': self.num_heads,
      })
      return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, pretrained=False, weights=False, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        if not pretrained:
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim
          )
        else:
          # pre-trained
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim, weights=[weights]
          ) 

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
      
    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'sequence_length': self.sequence_length,
      'vocab_size': self.vocab_size,
      'embed_dim': self.embed_dim,
      })
      return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'embed_dim': self.embed_dim,
      'latent_dim': self.latent_dim,
      'num_heads': self.num_heads,
      })
      return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [21]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-06-08 21:43:13--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-08 21:43:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-08 21:43:13--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [23]:
def get_weights(vectorizer, embeddings_path, is_gensim=False):
    path_to_glove_file = embeddings_path
    
    if is_gensim:
        embeddings_index = gensim.models.Word2Vec.load(embeddings_path)
        
    else:
        embeddings_index = {}
        with open(path_to_glove_file) as f:
            for line in f:
                word, coefs = line.split(maxsplit=1)
                coefs = np.fromstring(coefs, "f", sep=" ")
                embeddings_index[word] = coefs
    
    if not is_gensim:
        print("Found %s word vectors." % len(dict(embeddings_index)))
    else:
        print("Found %s word vectors." % (embeddings_index.wv.vectors.shape[0]))

    
    
    vocab = vectorizer.get_vocabulary()
    word_index = dict(zip(vocab, range(len(vocab))))
    num_tokens = len(vocab)
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        if not is_gensim:
            embedding_vector = embeddings_index.get(word)
        else:
            if word in embeddings_index.wv:
                embedding_vector = embeddings_index.wv[word]
            else:
                embedding_vector = None
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            embedding_matrix[i] = np.random.uniform(-.1, .1, size=(embedding_dim))
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))
    return embedding_matrix

In [24]:
english_embeddings = get_weights(eng_vectorization, './glove.6B.300d.txt')

Found 400000 word vectors.
Converted 11896 words (1268 misses)


In [25]:
! wget https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_wiki.zip
! unzip -q full_grams_cbow_300_wiki.zip

--2021-06-08 21:46:43--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_wiki.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1491895880 (1.4G) [application/zip]
Saving to: ‘full_grams_cbow_300_wiki.zip’

full_grams_cbow_300 100%[===================>]   1.39G  39.1MB/s    in 38s     

2021-06-08 21:47:21 (37.5 MB/s) - ‘full_grams_cbow_300_wiki.zip’ saved [1491895880/1491895880]



In [26]:
arabic_embeddings = get_weights(ar_vectorization, './full_grams_cbow_300_wiki.mdl', is_gensim=True)

Found 662109 word vectors.
Converted 12646 words (7354 misses)


In [27]:
ar_vocab_size = len(ar_vectorization.get_vocabulary())
en_vocab_size = len(eng_vectorization.get_vocabulary())
en_vocab_size, ar_vocab_size

(13164, 20000)

In [28]:
embed_dim = 300
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, en_vocab_size, embed_dim, pretrained=True, weights=english_embeddings)(encoder_inputs)
# x = PositionalEmbedding(sequence_length, num_tokens, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim, pretrained=True, weights=arabic_embeddings)(decoder_inputs)
# x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(ar_vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [30]:
googledrive_path = '/content/drive/MyDrive/Transformers/final/'

In [31]:
from keras import callbacks
early_stopping_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(googledrive_path+'/weights_adam.ckpt', monitor='val_accuracy', save_weights_only=True,verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=googledrive_path+"/logs")
lr_schr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=True, factor=0.3, min_lr=0.0001)
cbs = [early_stopping_cb, checkpoint_cb, tensorboard_callback, lr_schr]

In [32]:
epochs = 100  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding (Positiona (None, None, 300)    3964200     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder (Transforme (None, None, 300)    4119848     positional_embedding[0][0]       
________________________________________________________________________________________

In [ ]:
latest = tf.train.latest_checkpoint(googledrive_path)
transformer.load_weights(latest)

In [33]:
ar_vocab = ar_vectorization.get_vocabulary()
ar_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = sequence_length


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in val_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts[:30])
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\n', translated)
    print('*'*50)

I guess everything's okay because nothing comes up. 
 [start] اعتقد ان كل شيء انتهي كل شيء انتهي [end]
**************************************************
I think once your friend, Mr. Caldwell, understands the situation, he'll fill in some missing pieces. 
 [start] اعتقد ان ال وقت ال سيد كودويل سيتصل بك ال جحيم و ال نهاية [end]
**************************************************
What are you doing? 
 [start] ماذا تفعل ؟ [end]
**************************************************
It's that woman. 
 [start] انها تلك ال مراة [end]
**************************************************
The papers. What's the idea of throwing them out? 
 [start] ال ذي ما ال فكرة ؟ [end]
**************************************************
That has not been part of my teaching. 
 [start] هذا لن [UNK] من ال جزء من ال خاصة به ال خاصة به ال خاصة به ال خاصة [end]
**************************************************
He died before they reached port, not before the skipper had a description of the island and got an idea of wh

In [34]:
def get_bleu():
  
  preds, src = [], []

  with tqdm(total=len(val_pairs), position=0, leave=True) as pbar:
    for en_sent, ar_sent in tqdm(val_pairs, position=0, leave=True):
      translated = decode_sequence(en_sent)
      preds.append(translated)
      src.append(ar_sent)
      pbar.update()

    return src, preds
    # print_scores(src, preds)



In [58]:
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
def print_scores(trgs, preds):
    print('----- Bleu-n Scores -----')
    print("1:", corpus_bleu(trgs, preds, weights=[1.0/1.0])*100)
    print("2:", corpus_bleu(trgs, preds, weights=[1.0/2.0, 1.0/2.0])*100)
    print("3:", corpus_bleu(trgs, preds, weights=[1.0/3.0, 1.0/3.0, 1.0/3.0])*100)
    print("4:", corpus_bleu(trgs, preds)*100)
    print('-'*25)
    print('----- Rouge Scores -----')
    rouge = Rouge()
    scores = rouge.get_scores(preds, src, avg=True)
    for key, item in scores.items():
      print(key,':',item)

In [36]:
src, preds = get_bleu()

100%|██████████| 5267/5267 [20:57<00:00,  4.19it/s]


In [59]:
print_scores(src, preds)

----- Bleu-n Scores -----
1: 42.716420044320536


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


2: 65.35779987447599
3: 75.31213326921176
4: 80.84417101713393
-------------------------
----- Rouge Scores -----
rouge-1 : {'f': 0.4937869320226216, 'p': 0.5281938079866801, 'r': 0.48853570740613916}
rouge-2 : {'f': 0.16773662900926523, 'p': 0.17706054602896212, 'r': 0.1660911175213441}
rouge-l : {'f': 0.5046094857087982, 'p': 0.5483246760158327, 'r': 0.4873552657643637}
